In [ ]:
# Importamos las bibliotecas necesarias para manejar datos, hacer peticiones web y controlar el tiempo.
import re
import pandas as pd
import requests
import time

In [ ]:
# Configuración inicial para usar en las clases.
genres = ["Drama"]
start_year = 2010
end_year = 2025
headers = {
#"X-RapidAPI-Key":"2a71737446mshd130aebe5b852b0p1d4674jsn1c93e1eccde5", #Mábel1
#"X-RapidAPI-Key":"8ce0139101msh32b3fde4a9a06cep1a18ddjsneec758e62128", #Mábel2
#"X-RapidAPI-Key":"af196f0478mshd90862dc657e127p18d90bjsn598fe178eab7", #Mábel3
#"X-RapidAPI-Key":"7afebd53f4msh5a5378a53b56ed5p11796bjsn19ceecfe1a26", #Marga1
#"X-RapidAPI-Key":"431e779057msh1f0c87c2937a329p125efcjsn45064e4b4925", #Marga2
#"X-RapidAPI-Key":"2a71737446mshd130aebe5b852b0p1d4674jsn1c93e1eccde5", #Marga3
#"X-RapidAPI-Key":"c5dae3bc7cmsh99f5150b0f14271p1276dfjsne7487416c47c", #Gloria1
#"X-RapidAPI-Key":"b0262d2945msh1f1577867db30c6p12fca9jsnc5eb4666d6bb", #Gloria2
"X-RapidAPI-Key":"1ef9312677msh54e21139cb02deap1a8f37jsn53623375f3fe", #Paola1
#"X-RapidAPI-Key": "28a3c3ab32msha5ca6e00170b6c7p197d01jsn44c7f725210a", #Paola2
#"X-RapidAPI-Key":"2583d39c0emshe321f51d17d9328p10cfb0jsn38ed4dc93da2", #Silvia1
#"X-RapidAPI-Key":"47608e078dmsha9428889aafcc0ap1afc77jsn6e04b24d521c", #Silvia2
"X-RapidAPI-Host":"moviesdatabase.p.rapidapi.com"
}

In [ ]:
# Definimos una clase para manejar la recogida de datos de películas.
class MovieData:
    # Función inicial para configurar los parámetros básicos.
    def __init__(self, headers, genres, start_year, end_year):
        self.headers = headers  # Información para acceder a la API.
        self.genres = genres  # Tipos de películas que queremos buscar.
        self.start_year = start_year  # Año inicial.
        self.end_year = end_year  # Año final.
        self.sleep_time = 2  # Tiempo de espera inicial cuando la API limita las peticiones.

    # Función para buscar las películas en la API.
    def fetch_movies(self):
        movies_list = []  # Lista vacía para guardar los datos de las películas.
        # Bucles anidados para iterar por género, año y página.
        for genre in self.genres:
            for year in range(self.start_year, self.end_year + 1):
                for pagina in range(1, 51):
                    success = False  # Control para saber si la petición fue exitosa.
                    intentos = 0  # Contador de intentos de la petición.
                    while not success and intentos < 5:  # Si no es exitoso y tiene menos de 5 intentos.
                        url = f"https://moviesdatabase.p.rapidapi.com/titles?year={year}&genre={genre}&page={pagina}&limit=50"
                        response = requests.get(url, headers=self.headers)  # Hacemos la petición a la API.
                        # Comprobamos el código de estado de la respuesta.
                        if response.status_code == 200:
                            movies_data = response.json()["results"]  # Obtenemos los datos.
                            movies_list.extend(self.process_results(movies_data, genre))  # Procesamos y añadimos a la lista.
                            print(f"Recibiendo datos de {genre} en {year}, página {pagina}")
                            success = True  # Marcamos como exitoso.
                        elif response.status_code == 429:
                            print("Durmiendo... Demasiadas peticiones.")  # La API limita por demasiadas peticiones.
                            time.sleep(self.sleep_time * (2 ** intentos))  # Esperamos más tiempo en cada intento.
                            intentos += 1  # Incrementamos el contador de intentos.
                        else:
                            sms_error = response.json().get("error")  # Obtenemos mensaje de error.
                            print(f"Error tipo: {sms_error}")
                            break  # Salimos del bucle si hay otro tipo de error.
                        print("✅")
        return movies_list  # Devolvemos la lista de películas.

    # Función para procesar los datos obtenidos.
    def process_results(self, movies_data, genre):
        genre_movies = []  # Lista para guardar películas de un género.
        for data in movies_data:
            # Filtramos por tipo de título.
            if data["titleType"]["text"] in ["Movie", "Short"]:
                release_date = data.get("releaseDate")
                # Comprobamos que exista fecha de lanzamiento.
                if release_date and release_date.get("year") and release_date.get("month"):
                    movie_info = (
                        release_date.get("month"),
                        genre,
                        data["titleType"]["text"],
                        data["titleText"]["text"],
                        release_date.get("year"),
                        data["id"]
                    )
                    genre_movies.append(movie_info)  # Añadimos la info a la lista.
        return genre_movies  # Devolvemos las películas de un género.

# Clase para procesar y guardar los datos de las películas.
class MovieDataProcessor:
    def __init__(self, movies_list):
        self.movies_list = movies_list  # Lista de películas.

    # Función para convertir la lista en un dataframe.
    def to_dataframe(self):
        df = pd.DataFrame(self.movies_list, columnsy=["IMDB_ID", "Type", "Genre", "Movie_Title", "Release_Month", "Year"])
        return df  # Devolvemos el dataframe.

    # Función para guardar el dataframe en un archivo CSV.
    def save_to_csv(self, filename):
        df = self.to_dataframe()  # Obtenemos el dataframe.
        df.to_csv(filename, index=False)  # Guardamos el archivo sin índice.

In [ ]:
# Crear instancias de las clases y usarlas.
recoleccion = MovieData(headers, genres, start_year, end_year)
movies = recoleccion.fetch_movies()
infiniteplay = MovieDataProcessor(movies)

In [ ]:
# Crea el archivo CSV.
infiniteplay.save_to_csv(f'Películas_{genres}_{start_year}-{end_year}.csv')